I/Importation des modules et appel de l'API de Riot games

In [ ]:
import random
!pip install cassiopeia
import cassiopeia as cass
from bs4 import BeautifulSoup
cass.set_riot_api_key("clé") #Mettre le lien de la clé ici, je ne partage pas le mien

r='EUW'
"""
Voici la méthode de construction du dataset:

-On choisir un invocateur au hasard
-On regarde son historique et on sélectionne une partie classée
-On sauvegarde les win rate et maîtrises avec le champion joué de tous les participant
-On en sélectionne au hasard un autre invocateur
-On ajoute les données dans le dataset
-On recommance le procédé 

"""


II/Récupération  d'une liste de joueurs

In [ ]:
from cassiopeia import Summoner,Patch
from cassiopeia.data import Queue


def get_infos_summoners(summ):
  """
  On chosit une game et on récupère les ids des participants, 
  ainsi que leur teams
  """
  patch1 = Patch.from_str("12.18", region=r)
  patch2 = patch1
  summoner = Summoner(id=summ.id, region=r)
  match_history = cass.get_match_history(
        continent=summoner.region.continent,
        puuid=summoner.puuid,
        queue=Queue.ranked_solo_fives,
        start_time=patch1.start,
        end_time=patch2.end
        )
  
  #On prends une game au hasard dans l'historique
  #print("len(match_history) =",len(match_history))
  if len(match_history)==0:
    print("ça prends des douches ici")
    summ=random.choice(names[0])[0]
    return get_infos_summoners(summ)
  else:
    
    k=random.choice([i for i in range(len(match_history))])
    
    match = match_history[k]

    #On récupère les id des participants
    #p = match.participants[summoner] 
    p = match.participants
    invocateurs=[]
    i=0
    for p in match.blue_team.participants:       
      invocateurs.append([Summoner(id=p.summoner.id,region=r),
                          cass.Champion(id=p.champion.id,region=r),
                          i])
      i+=1
    for p in match.red_team.participants:       
      invocateurs.append([Summoner(id=p.summoner.id,region=r),
                          cass.Champion(id=p.champion.id,region=r),
                          i])
      i+=1
      #On vérifie qui a gagné
    win_blue=match.blue_team.win
      #On renvoi la liste des invocateurs et les vaincqueurs

    print("UNE LISTE DE SUMMS SORTIE")
    return(invocateurs,win_blue)

III/Récupération des rôles joués et réécriture des pseudos

In [ ]:

!apt update
!apt install chromium-chromedriver
!pip install selenium

from selenium import webdriver
import time

champions=cass.Champions(region = r)

def get_nom_sum(nom):
    """
    On utilise cette fonction afin de gérer l'existance d'espace dans un nom d'invcateurs,
    ce qui change l'url de la page
    """
    for k in range(len(nom)):
        if nom[k]==' ' and k!=0:
            nom[k]='%'
        return nom

def get_role(place):
    """
    selon la place dans la liste, On attribue le rôle du sumonner
    J'ai essayé d'utiliser les attributs "rôle" et "lane", mais ils sont
    soit déternminés par le rôle au départ (qui peut changer assez souvent)
    soit par l'endroit où on a le plus passé de temps qui souvent pour les 
    jungle et support n'est pas l'endroit attribué
    """
    if place==0 or place==5:
        return 'TOP'
    if place==1 or place==6:
        return 'JUNGLE'
    if place==2 or place==7:
        return 'MID'
    if place==3 or place==8:
        return 'CARRY'
    if place==4 or place==9:
        return 'SUPPORT'

IV/Récupération des win rates et maîtrises

In [ ]:
def get_win_rate_and_masteries(summ,champ,place,erreur):
    """
    On scrappe le site mobalitycs.gg pour récupérer les win rates d'un certain champion à un certain rôles
    et ce pour chaque invocateur. On récupères également les masteries sur le champs
    """
    if type(summ)==str:
        name=summ
        print("ENCORE OIT FUMIER")
    else:
        name = get_nom_sum(summ.name)
        
    role=get_role(place)
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    url = 'https://app.mobalytics.gg/lol/profile/euw/'+name+'/champion-pool?c_champion='+str(champ.id)+'&c_queue=RANKED_SOLO&c_role='+role
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    time.sleep(3)
    content = driver.page_source.encode('utf-8').strip()
    soup = BeautifulSoup(content,"html.parser")
    #names = soup.find_all("a",{"target":"blank"})
    wr = soup.find('div',{"class":"m-srjk1z e1cltcfb0"})
    driver.close()
    if wr==None:
        """
        Si on ne trouve rien, c'est que l'invocateur a changé de pseudo depuis la game récupérée
        On va donc chercher le pseudo qu'il portait dans la dernière game de son historique
        Avec une reconnaissance de l'id, qui lui, ne change jamais
        """
        
        if type(erreur)==str:
            """
            On est entré deux fois ici,on peut en conclure qu'il y a une erreur
            et on ne rouvera pas le wr du sommoner à ce rôle pour ce champion
            """
            print("étape X :C LA CATA(RINA)")
            summ=Summoner(name=summ,region=r)
            cm = cass.get_champion_mastery(summoner=erreur,champion=champ.id,region=r).points
            return [0,cm]

        H=cass.get_match_history(continent=summ.region.continent,puuid=summ.puuid)
        M=H[0]
        for k in M.participants:
            J=k.to_dict()
            if J['summonerId']==summ.id:
                ancien_nom =name
                print("ancien nom = ",ancien_nom)
                name=(J['summonerName'])
                print("nouveau nom = ",name)
                return get_win_rate_and_masteries(name,champ,place,ancien_nom)
    
    wr=wr.get_text() #On obtient ici le win rate en chaine de caractères
    if type(summ)==str:
        print("name = ",name)
        print("ancien nom =",erreur)
        summ=Summoner(name=erreur,region=r)
        print("le summoner est : ",summ.name)
        print("id summ = ",summ.id)
    cm = cass.get_champion_mastery(summoner=summ,champion=champ.id,region=r).points
    print("étape FINALE : IL Y A DES MASTERIES")
    if str(wr)[:3]=='100':
        return [100,cm]
    if str(wr)[1]=='.':
        print("wr = ",int(wr[0]))
        return([int(wr[0]),cm])
        
    else:
        print("wr = ",int(wr[:2]))
        return([int(wr[:2]),cm])

V/Création des listes à ajouter au dataset

In [ ]:
def data_game(game):
  """
  On prend un get_infos_summoners(sum) en antrée 
  et on en ressort les stats sur les champions des joueurs
  """
  win_rates_indiv=[] #On récupère les win rates de chacun ici
  masteries_indiv=[] #On récupère les masteries de chacun ici
  wr_blue,wr_red=0,0 #On les somme ici
  mas_blue,mas_red=0,0
  
  i=0
  for joueur in game[0][:5]:
    print("Nom , Champ, place = ",joueur[0].name,joueur[1].name,get_role(joueur[2]))
    f=get_win_rate_and_masteries(joueur[0],joueur[1],joueur[2],0)
    wr_blue+=f[0]
    mas_blue+=f[1]
    i+=1
    win_rates_indiv.append(f[0])
    masteries_indiv.append(f[1])
    print('joueur numéro ',i," sorti")
  for joueur in game[0][5:]:
    print("Nom , Champ, place = ",joueur[0].name,joueur[1].name,get_role(joueur[2]))
    f=get_win_rate_and_masteries(joueur[0],joueur[1],joueur[2],0)
    wr_red+=f[0]
    mas_red+=f[1]
    i+=1
    win_rates_indiv.append(f[0])
    masteries_indiv.append(f[1])
    print('joueur numéro ',i," sorti")
  
  
  win_blue=game[1]
  
  res = [win_rates_indiv,masteries_indiv,wr_blue,wr_red,mas_blue,mas_red,win_blue]
  return res

VI/Lancement de la boucle

In [ ]:
premier_sum = Summoner(name="Ferrettt", region=r)
#On commence avec lui
sum=premier_sum

RES=[]
  
for k in range(1500):
  names=get_infos_summoners(sum)
  RES.append(data_game(names))
  sum=random.choice(names[0])[0]
  print("On passe à l'itération numéro ",k+1)

VII/Création et formatage du dataframe avec les données récupérées et création d'un fichier excel

In [ ]:
import pandas as pd

df = pd.DataFrame(columns=['wr_joueur_1',
                           'wr_joueur_2',
                           'wr_joueur_3',
                           'wr_joueur_4',
                           'wr_joueur_5',
                           'wr_joueur_6',
                           'wr_joueur_7',
                           'wr_joueur_8',
                           'wr_joueur_9',
                           'wr_joueur_10',
                           'masteries_joueur_1',
                           'masteries_joueur_2',
                           'masteries_joueur_3',
                           'masteries_joueur_4',
                           'masteries_joueur_5',
                           'masteries_joueur_6',
                           'masteries_joueur_7',
                           'masteries_joueur_8',
                           'masteries_joueur_9',
                           'masteries_joueur_10',
                           'wr équipe bleu',
                           'wr équipe Rouge',
                           'masteries_blue',
                           'mawteries_red' ,
                           'Victoire bleus'],
                    data=[[0 for k in range(25)]])



def reshape(RES):
    R=[0]
    for l in RES:
        #print("l = ",l)
        a,b=[],[]
        for i in range(len(l)):
            #print("i = ",i)
            if i==0 or i==1:
                #print("i = ",i)
                for u in l[i]:
                    #print("u = ",u)
                    a.append(u)
            else:
                #print("i = ",i)
                #print("RES[i]= ",l[i])
                b.append(l[i])
        
        R.append(a+b)
    R.pop(0)
    return R

def ad_df(df,RES):
    R=reshape(RES)
    u='00'
    for k in range(len(R)):
        df.loc[k]=R[k]
    df.to_excel('FINALE_PART_'+u+'.xlsx')

ad_df(df,RES)
len(RES)